In [25]:
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.model_selection import cross_val_score, KFold
from keras.models import Sequential
from sklearn.metrics import accuracy_score
from keras.wrappers.scikit_learn import KerasRegressor
seed = 42
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import layers
from keras.optimizers import RMSprop
from keras import backend as K
K.set_image_dim_ordering('th')

In [2]:

# import importlib
# import os
4077, -24.20391846],
       [ -3.06291199, -29.1925354 , -54.17045593, ..., -11.0681076 ,
        -25.03192139, -48.92904663],
       ..., 
       [ 12.9357605 ,   0.0642662 , -43.96864319, ...,  -5.27310944,
        -21.49401855, -50.50608826],
       [ -1.900383  , -20.00675201, -50.1171875 , ...,  -7.74325943,
        -23.73028564, -61.93417358],
       [ 16.03391647, -19.57675934, -28.65151978, ...,  -3.3750267 ,
        -10.91635132, -41.78013611]])
# def set_keras_backend(backend):

#     if K.backend() != backend:
#         os.environ['KERAS_BACKEND'] = backend
#         importlib.reload(K)
#         assert K.backend() == backend

# set_keras_backend("tensorflow")

Using TensorFlow backend.


In [2]:
data = pd.read_csv('keys.txt', delimiter="(",  header=None)
for i in range(5):
    data[i] = data[i].astype(str).str[:-1]
data.columns = ["image","Head","left_hand","steering_wheel","right_hand"]
data["Head_x"], data["Head_y"] = data['Head'].str.split(',', 1).str
data["left_hand_x"], data["left_hand_y"] = data['left_hand'].str.split(',', 1).str
data["steering_wheel_x"], data["steering_wheel_y"] = data['steering_wheel'].str.split(',', 1).str
data["right_hand_x"], data["right_hand_y"] = data['right_hand'].str.split(',', 1).str
data.drop("Head",axis=1,inplace=True)
data.drop("left_hand",axis=1,inplace=True)
data.drop("steering_wheel",axis=1,inplace=True)
data.drop("right_hand",axis=1,inplace=True)

In [3]:
data.head()

,image,Head_x,Head_y,left_hand_x,left_hand_y,steering_wheel_x,steering_wheel_y,right_hand_x,right_hand_y
0,img_56.jpg,192,178,461,126,443,219,587,321
1,img_63.jpg,333,135,636,56,581,171,637,383
2,img_118.jpg,242,134,539,161,540,240,633,398
3,img_120.jpg,163,132,479,110,440,213,592,304
4,img_138.jpg,192,185,514,105,490,213,588,407


In [4]:
data.to_csv("preprocessed1.csv",sep="\t")

In [5]:
data["Head_x"] = (data["Head_x"].astype(int)*150/640).astype(int)

In [6]:
data["left_hand_x"] = (data["left_hand_x"].astype(int)*150/640).astype(int)
data["steering_wheel_x"] = (data["steering_wheel_x"].astype(int)*150/640).astype(int)
data["right_hand_x"] = (data["right_hand_x"].astype(int)*150/640).astype(int)

In [7]:
data["Head_y"] = (data["Head_y"].astype(int)*150/480).astype(int)
data["left_hand_y"] = (data["left_hand_y"].astype(int)*150/480).astype(int)
data["steering_wheel_y"] = (data["steering_wheel_y"].astype(int)*150/480).astype(int)
data["right_hand_y"] = (data["right_hand_y"].astype(int)*150/480).astype(int)

In [8]:
X = data["image"].values
data.drop("image",axis=1, inplace=True)
Y = data.values

In [30]:
Y[0]

array([ 45,  55, 108,  39, 103,  68, 137, 100])

In [23]:
#for regression network

import os
import shutil
path_to_train_folder = '/Users/Shruthi/Downloads/state-farm-distracted-driver-detection/imgs/train'
subs = os.listdir(path_to_train_folder)
for sub in subs:
    sub_folder = os.path.join(path_to_train_folder, sub)
    for imag in X:
        if os.path.exists(os.path.join(sub_folder,imag)):
            shutil.copyfile(os.path.join(sub_folder,imag), 'reg_train_input/'+imag)
            
    

In [10]:
x = np.array([np.array(Image.open(fname)) for fname in "reg_train_input/"+X])

In [11]:
x = x.transpose(0,3,1,2)
x.shape

(499, 3, 150, 150)

In [26]:
'''def key_model():
    model=Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(3,150,150))),1
    model.add(Activation('relu'))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))

    # the model so far outputs 3D feature maps (height, width, features)
    model.add(Flatten()) # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64)) # 64 neurons
    model.add(Activation('relu'))
    model.add(Dense(64)) # 64 neurons
    model.add(Activation('relu'))
    model.add(Dense(32)) # 64 neurons
    model.add(Activation('relu'))

    # output layer: get 8 coordinate values
    model.add(Dense(8))
    model.compile(optimizer='rmsprop',
              loss='mse')
    return model
'''
model=Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(3,480,640))),1
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))

# the model so far outputs 3D feature maps (height, width, features)
model.add(Flatten()) # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64)) # 64 neurons
model.add(Activation('relu'))
model.add(Dense(64)) # 64 neurons
model.add(Activation('relu'))
model.add(Dense(32)) # 64 neurons
model.add(Activation('relu'))
model.add(layers.Dense(8))

model.compile(optimizer='rmsprop',loss='mse', metrics=['mae'])

In [27]:
k = 4
num_val_samples = 499 // k
num_epochs = 2
all_scores = []

In [31]:
for i in range(k):
    print(f'Processing fold # {i}')
    val_data = x[i * num_val_samples: (i+1) * num_val_samples]
    val_targets = Y[i * num_val_samples: (i+1) * num_val_samples]
    
    partial_train_data = np.concatenate(
                            [x[:i * num_val_samples],
                            x[(i+1) * num_val_samples:]],
                            axis=0)
    partial_train_targets = np.concatenate(
                            [Y[:i * num_val_samples],
                            Y[(i+1)*num_val_samples:]],
                            axis=0)
    res = model.fit(partial_train_data,
              partial_train_targets,
              epochs=num_epochs,
              batch_size=1,
              validation_data = (val_data,val_targets))
    print("Scores: ", res)

Processing fold # 0
Train on 375 samples, validate on 124 samples
Epoch 1/2
375/375 [==============================] - 623s 2s/step - loss: 8865281.9235 - mean_absolute_error: 216.8003 - val_loss: 9394.9597 - val_mean_absolute_error: 77.8138
Epoch 2/2
375/375 [==============================] - 581s 2s/step - loss: 691.9174 - mean_absolute_error: 19.2956 - val_loss: 155.1311 - val_mean_absolute_error: 9.7630
Scores:  <keras.callbacks.History object at 0x1a49760860>
Processing fold # 1
Train on 375 samples, validate on 124 samples
Epoch 1/2
375/375 [==============================] - 617s 2s/step - loss: 356.4052 - mean_absolute_error: 14.4864 - val_loss: 316.0217 - val_mean_absolute_error: 14.4293
Epoch 2/2
375/375 [==============================] - 603s 2s/step - loss: 252.3639 - mean_absolute_error: 12.2540 - val_loss: 175.3323 - val_mean_absolute_error: 10.7833
Scores:  <keras.callbacks.History object at 0x1a4aa44fd0>
Processing fold # 2
Train on 375 samples, validate on 124 samples
E

In [33]:
from keras.models import model_from_json
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")    

Saved model to disk
